Referenced papers:
- https://arxiv.org/pdf/2203.11171 (original self consistency paper)
- https://arxiv.org/pdf/2311.17311 (universal self consistency)

General Idea:
- Sample directly from the decoder instead of greedy decoding
- Create a bunch of different responses, find the most consistent result
- Return the most consistent result

Authors of the original paper use PaLM-540B

In [29]:
!pip install datasets
!pip install transformers
!pip install --upgrade transformers
!pip install einops
!pip install evaluate bert_score rouge-score
!pip install -U datasets huggingface_hub fsspec
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


...


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 2.1 MB/s eta 0:00:06
     ---- ----------------------------------- 1.6/12.8 MB 3.4 MB/s eta 0:00:04
     -------------- ------------------------- 4.7/12.8 MB 7.2 MB/s eta 0:00:02
     -------------------------------- ------ 10.7/12.8 MB 13.2 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 13.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached scipy-1.16.1-cp313-cp313-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.7 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/8.7 MB 3.3 MB/s eta 0:00:03
   ------------ --------------------------- 2.6/8.7 MB 5.2 MB/s eta 0:00:02
   ---------------------------------- ----- 7.6/8.7 MB 10.9 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 11.4 MB/s eta 0:00:00
Using cached scipy-1.16.1-cp313-cp313-win_amd64.whl (38.5 MB)



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset
import json

c:\Users\Ericb\Documents\TOE-Summ\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_dataset = load_dataset("abisee/cnn_dailymail", "1.0.0", split="train")
test_dataset = load_dataset("abisee/cnn_dailymail", "1.0.0", split="test")

In [4]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [5]:
!pip install huggingface_hub
!pip install -U "huggingface_hub[cli]"


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# OPENROUTER
import requests

API_KEY = "LOOK IN SLACK"
REFERER = "https://colab.research.google.com"

MODEL = "moonshotai/kimi-k2"
MAX_TOKENS = 400

# OpenRouter API endpoint
url = "https://openrouter.ai/api/v1/chat/completions"

# Headers required by OpenRouter
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "HTTP-Referer": REFERER,
    "Content-Type": "application/json"
}

In [8]:
import evaluate

rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

In [30]:
def evaluate_summary(gen: str, ref: str, source: str = None):
  """
  Takes:
    - gen: the models generated summary
    - ref: the human (reference) summary
    - source: the source text used to generate the summary
  Returns: a dict of metric_name -> score
  """

  results = {}


  # Rouge
  scores = rouge.compute(predictions=[gen], references=[ref], use_stemmer=True)
  print("S", scores)
  # Rouge.compute returns keys like rouge1 and rouge2 each as a dict ofprecision/recall/fmeasure
  # What is the v.mid.fmeasure for? The attribute doesn't exist for the given value (as it is a numpy float)

  # Bert score
  bs = bertscore.compute(predictions=[gen], references=[ref], lang="en")

  # bert score returns P, F, R Lists

  results["bertscore_f1"] = bs["f1"][0]

  # compression ratio (optional)
  if source:
    results["compression"] = len(gen.split()) / len(source.split())

  # Return ROUGE results + f1 scores
  results.update(**scores)
  return results

Prompts from https://arxiv.org/pdf/2407.21443

In [22]:
# Specify the labels for:
# (1) The text to summarize
# (2) The summarization of the text given by the dataset
# Ex. For dailymail, the text to summarize is given as the label "article", while the summarization is given as "highlights".
text_to_summarize_label = "article"
summarization_label = "highlights"

In [42]:
cot = ""

# No chain of thought for now
# for i in train_dataloader.dataset[0:3][summarization_label]:
#   cot += "Summary: " + i + "\n"

cot += "Summarize the article below in a total of 3 sentences. Do not put in bullet form, leave in a paragraph form."
cot += "Start your response with 'Response: ' (without quotes)."
def append_prompt(cot_to_append, prompt):
  return cot_to_append + "\n" + prompt + "\n"

In [25]:
# adjective sets
EXTRACTION_TYPES_TEXT_SUMM  = [ "insightful", "clear", "concise", "accurate",
                            "coherent", "persuasive", "thorough", "engaging",
                             "relevant", "structured", "nuanced", "well-supported",
                             "articulate", "convincing", "sophisticated", "balanced"]
# Decide scoring priority for each adjective here.
# Ex. If you want a consise response, you might want to set "conciseness": MAX.
# If you have something a bit more ambiguous, you might want to set the scores to different priorities (ex. "conciseness": MEDIUM, "detailedness": SMALL, "clarity": HIGH)
NONE = 0
LOW = 0.25
MEDIUM = 0.50
HIGH = 0.75
MAX = 1

EXTRACTION_TYPES_TEXT_SUMM_SCORES = {
    "insightful": {"conciseness": LOW, "detailedness": HIGH, "clarity": MEDIUM},
    "clear": {"conciseness": HIGH, "detailedness": LOW, "clarity": MAX},
    "concise": {"conciseness": MAX, "detailedness": NONE, "clarity": NONE},
    "accurate": {"conciseness": LOW, "detailedness": HIGH, "clarity": MEDIUM},
    "coherent": {"conciseness": MEDIUM, "detailedness": MEDIUM, "clarity": HIGH},
    "persuasive": {"conciseness": LOW, "detailedness": HIGH, "clarity": MEDIUM},
    "thorough": {"conciseness": NONE, "detailedness": MAX, "clarity": MEDIUM},
    "engaging": {"conciseness": LOW, "detailedness": HIGH, "clarity": MEDIUM},
    "relevant": {"conciseness": HIGH, "detailedness": MEDIUM, "clarity": MEDIUM},
    "structured": {"conciseness": MEDIUM, "detailedness": LOW, "clarity": HIGH},
    "nuanced": {"conciseness": LOW, "detailedness": HIGH, "clarity": MEDIUM},
    "well-supported": {"conciseness": LOW, "detailedness": MAX, "clarity": MEDIUM},
    "articulate": {"conciseness": MEDIUM, "detailedness": MEDIUM, "clarity": HIGH},
    "convincing": {"conciseness": MEDIUM, "detailedness": HIGH, "clarity": MEDIUM},
    "sophisticated": {"conciseness": LOW, "detailedness": HIGH, "clarity": MEDIUM},
    "balanced": {"conciseness": MEDIUM, "detailedness": HIGH, "clarity": MEDIUM}
}

# split into four tiers
EXTRACTION_SETS = {
    8:  EXTRACTION_TYPES_TEXT_SUMM[:8],
    16: EXTRACTION_TYPES_TEXT_SUMM[:16],
    24: EXTRACTION_TYPES_TEXT_SUMM[:24],
    32: EXTRACTION_TYPES_TEXT_SUMM[:32],
}

# Example usage:
for size, extraction_type in EXTRACTION_SETS.items():
    print(f"{size}-word set:", extraction_type)

8-word set: ['insightful', 'clear', 'concise', 'accurate', 'coherent', 'persuasive', 'thorough', 'engaging']
16-word set: ['insightful', 'clear', 'concise', 'accurate', 'coherent', 'persuasive', 'thorough', 'engaging', 'relevant', 'structured', 'nuanced', 'well-supported', 'articulate', 'convincing', 'sophisticated', 'balanced']
24-word set: ['insightful', 'clear', 'concise', 'accurate', 'coherent', 'persuasive', 'thorough', 'engaging', 'relevant', 'structured', 'nuanced', 'well-supported', 'articulate', 'convincing', 'sophisticated', 'balanced']
32-word set: ['insightful', 'clear', 'concise', 'accurate', 'coherent', 'persuasive', 'thorough', 'engaging', 'relevant', 'structured', 'nuanced', 'well-supported', 'articulate', 'convincing', 'sophisticated', 'balanced']


In [31]:
## IMPROVED SCORING FUNCTIONS
import math
import gzip
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import numpy as np

nlp = spacy.load("en_core_web_sm")

_FILLER_PATTERNS = [
    r'\bas a matter of fact\b', r'\bin order to\b', r'\bas well as\b',
    r'\bat the end of the day\b', r'\bin the event that\b', r'\bthe fact that\b',
    r'\bjust to be clear\b', r'\bkind of\b', r'\bsort of\b', r'\bvery important\b',
    r'\breally\b', r'\bactually\b', r'\bperhaps\b', r'\bpretty much\b', r'\bessentially\b'
]

def _count_fillers(text: str) -> int:
    lowered = text.lower()
    return sum(len(re.findall(p, lowered)) for p in _FILLER_PATTERNS)

def _compressed_size(text: str) -> int:
    return len(gzip.compress(text.encode('utf-8')))

def conciseness_score(cand: str, pool: list) -> float:
    """Improved conciseness scoring with better normalization"""
    if not cand or not cand.strip():
        return 0.0

    tokens = cand.split()
    length = len(tokens)

    if length == 0:
        return 0.0

    comp_size = _compressed_size(cand)
    info_density = comp_size / length

    # Use pool statistics for better baseline if available
    if pool and len(pool) > 0:
        valid_pool = [str(p) for p in pool if p and str(p).strip()]
        if valid_pool:
            pool_densities = [_compressed_size(p) / max(1, len(p.split())) for p in valid_pool]
            baseline = np.mean(pool_densities) if pool_densities else 10.0
        else:
            baseline = 10.0
    else:
        baseline = 10.0

    density_score = 1.0 - min(1.0, info_density / baseline)

    # Filler penalty
    filler_count = _count_fillers(cand)
    filler_ratio = filler_count / max(1, length / 5)
    filler_penalty = min(1.0, filler_ratio)

    score = density_score * (1 - 0.3 * filler_penalty)  # Reduced penalty weight
    return max(0.0, min(1.0, round(score, 4)))  # Ensure score is in [0,1]


def clarity_score(cand: str, pool: list) -> float:
    """
    FIXED: Improved clarity scoring that actually works
    """
    if not cand or not cand.strip():
        return 0.0

    # Handle empty pool case
    valid_pool = [str(p) for p in pool if p and str(p).strip()]

    if not valid_pool:
        # Fallback to readability-based scoring
        return _readability_score(cand)

    try:
        # Create documents list
        docs = valid_pool + [cand]

        # Use TF-IDF with better parameters (FIXED the typo: min_df not mind_df)
        vectorizer = TfidfVectorizer(
            stop_words="english",
            min_df=1,
            max_features=1000,
            ngram_range=(1, 2)  # Include bigrams for better context
        )

        X = vectorizer.fit_transform(docs)

        if X.shape[0] <= 1:
            return _readability_score(cand)

        # Get candidate vector (last one)
        cand_vec = X[-1]
        pool_vecs = X[:-1]

        # Calculate similarity to pool (mean of similarities to each reference)
        similarities = []
        for i in range(pool_vecs.shape[0]):
            sim = cosine_similarity(cand_vec, pool_vecs[i:i+1])[0, 0]
            similarities.append(sim)

        avg_similarity = np.mean(similarities) if similarities else 0.0

        # Gentler length penalty
        length = len(cand.split())
        length_penalty = 1 / (1 + 0.05 * math.log1p(length))  # Much gentler penalty

        final_score = avg_similarity * (0.7 + 0.3 * length_penalty)  # Weighted combination

        return max(0.0, min(1.0, round(final_score, 4)))

    except Exception as e:
        print(f"Error in clarity_score: {e}")
        return _readability_score(cand)

def _readability_score(text: str) -> float:
    """Fallback readability score based on sentence structure"""
    if not text.strip():
        return 0.0

    sentences = re.split(r'[.!?]+', text)
    sentences = [s.strip() for s in sentences if s.strip()]

    if not sentences:
        return 0.0

    # Average sentence length (penalize very long or very short sentences)
    avg_sent_len = np.mean([len(s.split()) for s in sentences])
    length_score = 1.0 / (1.0 + abs(avg_sent_len - 15) / 10)  # Optimal around 15 words

    # Vocabulary diversity
    words = text.lower().split()
    unique_words = len(set(words))
    diversity_score = min(1.0, unique_words / max(1, len(words)))

    return round(0.6 * length_score + 0.4 * diversity_score, 4)

def detail_score(cand: str, pool: list) -> float:
    """
    IMPROVED: Detail scoring with better handling and normalization
    """
    if not cand or not cand.strip():
        return 0.0

    try:
        doc = nlp(cand)

        # Content word density (improved)
        content_words = [t for t in doc if t.pos_ in {'NOUN', 'VERB', 'ADJ', 'ADV'} and not t.is_stop]
        total_tokens = len([t for t in doc if not t.is_space])
        density = len(content_words) / max(1, total_tokens)

        # Named entities and numbers (indicators of specific details)
        entities = len(doc.ents)
        numbers = len([t for t in doc if t.like_num])
        detail_indicators = (entities + numbers) / max(1, total_tokens)

        # Structural complexity
        sentences = list(doc.sents)
        num_sentences = len(sentences)
        if num_sentences > 0:
            avg_sent_complexity = np.mean([len([t for t in sent if not t.is_space]) for sent in sentences])
            structure_score = min(1.0, (num_sentences * avg_sent_complexity) / 100)
        else:
            structure_score = 0.0

        # Length contribution (but capped and gentler)
        length = len(doc)
        length_contrib = min(0.3, math.log1p(length) / 50)

        final_score = 0.4 * density + 0.3 * detail_indicators + 0.2 * structure_score + 0.1 * length_contrib

        return max(0.0, min(1.0, round(final_score, 4)))

    except Exception as e:
        print(f"Error in detail_score: {e}")
        # Fallback to simpler scoring
        words = cand.split()
        return min(1.0, len(words) / 100)  # Simple length-based fallback

def score_answer(answer, reference_summaries=None):
    """
    IMPROVED: Better scoring with proper reference handling
    """
    # Handle the case where reference_summaries might be a dataset slice
    ref_pool = []
    if reference_summaries is not None:
        if hasattr(reference_summaries, '__iter__') and not isinstance(reference_summaries, str):
            # It's likely a dataset slice or list
            ref_pool = [str(s) for s in reference_summaries if s and str(s).strip()]
        else:
            # It's a single string
            ref_pool = [str(reference_summaries)] if str(reference_summaries).strip() else []

    concise_score = conciseness_score(answer, ref_pool)
    clear_score = clarity_score(answer, ref_pool)  # Now uses the improved version
    det_score = detail_score(answer, ref_pool)

    scores = {
        "conciseness": concise_score,
        "clarity": clear_score,
        "detailedness": det_score
    }

    print(f"Scores: {scores}")
    return scores

def select_best_extraction_type(answer, reference_summaries=None):
    """
    IMPROVED: Better selection using weighted scoring instead of simple deviation
    """
    scores = score_answer(answer, reference_summaries)

    extraction_type_scores = {}

    for adj_type, target_scores in EXTRACTION_TYPES_TEXT_SUMM_SCORES.items():
        # Calculate weighted similarity instead of simple deviation
        total_similarity = 0
        total_weight = 0

        for dimension, target_value in target_scores.items():
            if dimension in scores:
                actual_value = scores[dimension]
                # Use gaussian similarity instead of absolute deviation
                deviation = abs(target_value - actual_value)
                weight = target_value if target_value > 0 else 0.1  # Weight by target importance
                similarity = math.exp(-2.0 * deviation**2)  # Gaussian similarity
                total_similarity += weight * similarity
                total_weight += weight

        if total_weight > 0:
            extraction_type_scores[adj_type] = total_similarity / total_weight
        else:
            extraction_type_scores[adj_type] = 0

    print(f"Extraction type scores: {extraction_type_scores}")

    # Return the type with highest similarity score
    best_type = max(extraction_type_scores.items(), key=lambda x: x[1])
    return best_type[0], extraction_type_scores

In [ ]:
#TO CHECK IF IT WORKS
from torch.utils.data import DataLoader
import re


# Note - there are better ways to do this than what is shown here. This is just for illustration, as a simple way to extract the intended generation.
def extract_intended_generation(full_text): # You will most likely need to create variations of this method to extract your intended answer, depending on the output of your prompt.
    try:
      print(full_text)
      # The output includes the prompt as well. Split the text into segments based on 'A: ' which starts each answer
      segments = full_text.split('Summary: ')
      print("SEGMENTS:" ,segments)
      # Take the last segment, which should be the most recent answer
      last_answer = segments[1].strip().split("Summary: ")[0]

      # For some reason, the outputted answer includes part of 'Question 1' afterwards. This code splits the output before that occurs
      #last_answer = last_answer.split("Question 1:")[0].strip()

      return last_answer

    except ValueError as e:
      return None



def extract_output_numerical_answer(text, key_phrase="The answer is"): # You will most likely need to create variations of this function to extract your intended numerical answer, depending on the output of your prompt.
    if text is None:
        return None

    try:
        # Find the position where 'The answer is' starts in the text
        start_pos = text.index(key_phrase) + len(key_phrase)

        # Extract the text that comes after 'The answer is'
        answer_text = text[start_pos:].strip()

        # Use a regular expression to find a floating point or integer number
        matches = re.findall(r'[-+]?\d*\.\d+|\d+', answer_text)

        if matches:
            return float(matches[0])
        else:
            return None

    except ValueError:
        return None


def extract_true_numerical_answer(text):
    # Find the position of the "#### " pattern in the text
    position = text.find("#### ")

    # Extract the numerical part that follows this pattern
    if position != -1:
        numerical_str = text[position + 5:].strip()
        try:
            return float(numerical_str)
        except ValueError:
            print(f"Error: Could not convert '{numerical_str}' to float.")
            return None
    else:
        return None  # or some error handling

Note: All the functions for running each algorithm (toe and greedy) will clean the output INSIDE the function.

In [44]:
#Implemented Task-based Output Extraction(toe)
#to make it UNIVERSAL SELF CONSISTENCY, extraction_type = "consistent"
# No COT for text summ
def output_extraction(question, num_samples, responses, extraction_type):
    joined_responses = "\n".join([f"Response {i+1}:\n{resp}" for i, resp in enumerate(responses)])
    selection_prompt = (
        f"I have generated  the following {num_samples} responses to the question: {question}\n"
        f"Select the most {extraction_type} response based on majority consensus..\n"
        f"Respond ONLY with the response number.\n\n"
        f"{joined_responses}"
    )

    payload = {
    "model": MODEL,
    "max_tokens": MAX_TOKENS,
    "messages": [{"role": "user", "content": selection_prompt}],
    "temperature": 0
    }
    response = requests.post(
        "https://openrouter.ai/api/v1/chat/completions", 
        headers={
            "Authorization": f"Bearer {API_KEY}",
            "HTTP-Referer": REFERER,
            "Content-Type": "application/json"
        }, json=payload)
    if response.status_code == 200:
        selection_text = response.json()["choices"][0]["message"]["content"]
        selected_num = None
        try:
            selected_num = int(selection_text[-1].rstrip('.'))
        except ValueError:
            print("Error for: ", selection_text)
        final_answer = None
        if selected_num is not None and 1 <= selected_num <= len(responses):
            final_answer = responses[selected_num - 1]
        return final_answer
    else:
        print(f"WARNING: Response failed with code {response.status_code}")

In [45]:
# Note: If you want to test the whole dataset, you can change test_dataloader to train_dataloader (usually contains more samples) and remove the indexing
def run_toe(question, num_samples, extraction_type=None): #inputs is an already tokenized question
  temperature = 0.8
  payload = {
    "model": MODEL,
    "max_tokens": MAX_TOKENS,
    "temperature": temperature,
    "messages": [
        {"role": "user", "content": question}
    ]}

  #we sample on our own
  sampled_outputs = []
  for i in range(0, num_samples):
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200: #check if it processed it and gave a result
      data = response.json()
      sampled_outputs.append(data["choices"][0]["message"]["content"])
      print(f"Samples {len(sampled_outputs) - 1}: ", sampled_outputs[-1])
    else:
      print("Error:", response.status_code, response.text)

  extraction_type_frequency = {}
  for i, output in enumerate(sampled_outputs):
    intended_output = extract_intended_generation(output) # extract_intended_generation(output), for some reason the extraction doesn't work? (The model isn't adding "Summary: " at the beginning, should be solved with better models)
    sampled_outputs[i] = intended_output

    # If we want automatic selection
    if extraction_type is None:
      extraction_types = {}
      scores = score_answer(intended_output) # Returns {"conciseness": X, "detailedness": Y, "clarity": Z}
      for k, v in EXTRACTION_TYPES_TEXT_SUMM_SCORES.items():
        conciseness_dev = abs(v["conciseness"] - scores["conciseness"])
        detailedness_dev = abs(v["detailedness"] - scores["detailedness"])
        clarity_dev = abs(v["clarity"] - scores["clarity"])

        # Get average deviations
        total_dev = [conciseness_dev, detailedness_dev, clarity_dev]
        average_dev = sum(total_dev) / len(total_dev)
        extraction_types[k] = average_dev

      # Get the adjective with the least deviation
      least_dev = {k:v for k,v in extraction_types.items() if v == min(list(extraction_types.values()))}
      extract_type = list(least_dev.keys())[0]
      extraction_type_frequency[extract_type] = extraction_type_frequency.get(extract_type, 0) + 1

  # Get the first extraction type that has the highest frequency (ie. most responses are best summarized in this manner)
  if extraction_type is None:
    extraction_type = max(extraction_type_frequency, key=extraction_type_frequency.get)
    extracted_output = output_extraction(question, sampled_outputs, extraction_type)
    return extracted_output, extraction_type
  else:
    extracted_output = output_extraction(question, sampled_outputs, extraction_type)
  #output_extraction takes sampled outputs and return one extracted output based on an extraction type
  print("extraction type: ", extraction_type, " answer: ", extracted_output)

  return extracted_output

In [35]:
# Note: If you want to test the whole dataset, you can change test_dataloader to train_dataloader (usually contains more samples) and remove the indexing
def run_greedy(question):
  payload = {
  "model": MODEL,
  "max_tokens": MAX_TOKENS,
  "messages": [{"role": "user", "content": question}],
  "temperature": 0
  }
  response = requests.post(
      "https://openrouter.ai/api/v1/chat/completions", 
      headers={
          "Authorization": f"Bearer {API_KEY}",
          "HTTP-Referer": REFERER,
          "Content-Type": "application/json"
      }, json=payload)
  if response.status_code == 200:
      selection_text = response.json()["choices"][0]["message"]["content"]
      return extract_intended_generation(selection_text)
  else:
      print(f"WARNING: Response failed with code {response.status_code}")

In [48]:
print(train_dataloader.dataset[20][text_to_summarize_label])
print("--------------------------------------------------------------------------------------------------------------------")
for i in range(1):
  output = (run_toe(append_prompt(cot, train_dataloader.dataset[i][text_to_summarize_label]), 5))

LAS VEGAS, Nevada (CNN)  -- Former football star O.J. Simpson will be held without bail after his arrest on robbery and assault charges, police announced late Sunday. Police released this mug shot of O.J. Simpson after his arrest. Simpson is accused of having directed several other men in an alleged armed robbery of sports memorabilia in a room at a Las Vegas hotel room. Las Vegas authorities said they have no information leading them to believe Simpson was carrying a firearm during the alleged incident at the Palace Station Hotel and Casino. Police said Simpson and other men burst into the room and walked out with the memorabilia, including some that was unrelated to Simpson, police said. "We don't believe that anyone was roughed up, but there were firearms involved," Lt. Clint Nichols told reporters. Nichols said the firearms were pointed at the victims. A reporter asked Nichols: Was "O.J. was the boss in that room?" Nichols responded, "That is what we believe, yes."  Watch Simpson t

IndexError: list index out of range

In [ ]:
print(train_dataloader.dataset[15][summarization_label] + "\n")

NEW: President Bush says he and first lady are deeply saddened by the tragedy . Mine Safety and Health Administration chief: We've run out of options. The six men have been trapped underground since August 6 . Seven bore holes drilled into the mountain have found no signs of life .



In [ ]:
# Removed redundant evaluation
# score = evaluate_summary(output[0], train_dataloader.dataset[15][summarization_label])
# print(score)

In [ ]:
print(score)

{'bertscore_f1': 0.8339174389839172, 'rouge1': np.float64(0.04597701149425287), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.04597701149425287), 'rougeLsum': np.float64(0.04597701149425287)}


In [ ]:
import csv
import os
import json

def save_result_incrementally(result, file_path, extraction_type): # result is a python dictionary containing data that will become clearer in the next section.
    # with open(file_path, 'a', newline='') as file:
    with open(file_path, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Ensure all keys are present in the result dictionary before writing
        row_data = [result.get('question_num', ''), result.get('question', ''), result.get('true_answer', ''), result.get(f'{extraction_type}_answer', ''), result.get("extraction_type", "")]
        writer.writerow(row_data)


def csv_to_json(csv_file_path, json_file_path, extraction_type):
    if os.path.exists(csv_file_path):
      results = []
      with open(csv_file_path, mode='r') as file:
          csv_reader = csv.reader(file)

          for row in csv_reader:
              # Ensure row has enough columns before accessing
              if len(row) > 3:
                  result = {
                      "question_num": row[0],
                      "question": row[1],
                      "true_answer": row[2],
                      f'{extraction_type}_summary': row[3]
                  }
                  results.append(result)

      with open(json_file_path, 'w') as json_file:
          json.dump(results, json_file, indent=4)


def load_results_from_json(json_file_path):
    results = []
    if os.path.exists(json_file_path):
      with open(json_file_path, 'r') as f:
          results = json.load(f)
    return results

task_type = 'summarization'
model_name = "phi1.5b"
dataset_name = "billsum"
base_dir = '/content/drive/My Drive/WKER/toe-results'

In [ ]:
# Saves all text summarization metrics
def save_last_question_data(total_questions, summary_scores, file_path):
  with open(file_path, 'w') as file:
    file.write(f'{total_questions} {summary_scores["bertscore_f1"]} {summary_scores["rouge1"]} {summary_scores["rouge2"]} {summary_scores["rougeL"]} {summary_scores["rougeLsum"]}')

def last_question_stats(file_path):
  total_questions = 0
  f1_score = 0
  rouge1, rouge2, rougeL, rougeLsum = 0, 0, 0, 0
  if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
    with open(file_path, 'r') as file:
      line = file.readline()
      print(line)
      total_questions, f1_score, rouge1, rouge2, rougeL, rougeLsum = map(float, line.strip().split())
  return total_questions, f1_score, rouge1, rouge2, rougeL, rougeLsum

In [ ]:
def path_based_on_extraction_type(extraction_type, flexible=False):
  if flexible:
    our_dir_name_base_filename = f'{model_name}_{dataset_name}_flexible'
  else:
    our_dir_name_base_filename = f'{model_name}_{dataset_name}_{extraction_type}'

  our_dir_file_path = os.path.join(base_dir, task_type, our_dir_name_base_filename)
  if not os.path.exists(our_dir_file_path):
    os.makedirs(our_dir_file_path)

  last_question_stats_base_filename = f'{extraction_type}_last_question_stats'
  last_question_stats_file_path = os.path.join(our_dir_file_path, last_question_stats_base_filename + '.txt')

  csv_base_filename = f'{extraction_type}' # Be organized in how you name it, don't just name it "temp1" or something like that. Here's a sample name.
  csv_file_path = os.path.join(our_dir_file_path, csv_base_filename + '.csv')

  json_base_filename = f'{extraction_type}'
  json_file_path = os.path.join(our_dir_file_path, json_base_filename + '.json')

  return last_question_stats_file_path, csv_file_path, json_file_path #in last question stats we have total number of questions and correct answers

In [ ]:
num_of_questions = 8
num_samples = 8

In [ ]:
# This function seems useful when we want to evaluate the whole dataset, but if you want to just get the samples, stick to the experiment function
def run_summarization_baseline(dataset, source_key, ref_key, summarizer_fn, extraction_type):
    metrics_to_track = ["rouge1","rouge2","rougeL","bertscore_f1","compression"]
    all_metrics = {m: [] for m in metrics_to_track}
    for ex in dataset:
        src = ex[source_key]
        ref = ex[ref_key]
        if extraction_type == "greedy":
          gen = summarizer_fn(src)
        else:
          gen = summarizer_fn(src, num_samples=num_samples, extraction_type=extraction_type)
        sc  = evaluate_summary(gen, ref, source=src)
        for m in metrics_to_track:
            all_metrics[m].append(sc[m])
    return {m: sum(v)/len(v) for m,v in all_metrics.items()}

In [ ]:
def experiment(extraction_type, num_samples=None): #either greedy, sc, or a type of adjective for extraction_type (detailed, consistent)
    #loading path to data about 1. last question stats (how many total questions and correct answers) 2. csv file and 3. json file
    last_question_stats_file_path, csv_file_path, json_file_path = path_based_on_extraction_type(extraction_type)
    total_questions, f1_score, rouge1, rouge2, rougeL, rougeLsum = last_question_stats(last_question_stats_file_path) # ALL TOTAL SCORES
    total_questions = int(total_questions)
    results = [] #results that we have for greedy decoding

    # f1_scores = []
    # rouge1_scores = []
    # rouge2_scores = []
    # rougeL_scores = []
    # rougeLsum_scores = []

    csv_to_json(csv_file_path, json_file_path, extraction_type)
    results = load_results_from_json(json_file_path)
    true_answers = train_dataloader.dataset[total_questions:num_of_questions+total_questions][summarization_label]

    for i, question in enumerate(train_dataloader.dataset[total_questions:num_of_questions+total_questions][text_to_summarize_label]):
        # here we use few shot COT
        extract_type = None
        appended_question = append_prompt(cot, question)

        if extraction_type == "greedy":
          output_summary = run_greedy(appended_question)
        elif extraction_type == "flexible":
          output_result = run_toe(appended_question, num_samples=num_samples)
          output_summary = output_result[0]  # Extract the summary string
          extract_type = output_result[1] # Extract the extraction type
        else:
          output_summary = run_toe(appended_question, num_samples=num_samples, extraction_type=extraction_type)

        summary_score = evaluate_summary(output_summary, true_answers[i]) # Use output_summary here
        result = {
            "question_num": total_questions,
            "question": question,
            "true_answer": true_answers[i],
            f"{extraction_type}_answer": output_summary,
            f"extraction_type": extract_type
        }
        result = {k: v for k, v in result.items() if v != None}
        results.append(result)
        print(result)

        f1_score += summary_score["bertscore_f1"]
        rouge1 += summary_score["rouge1"]
        rouge2 += summary_score["rouge2"]
        rougeL += summary_score["rougeL"]
        rougeLsum += summary_score["rougeLsum"]

        summary_score["bertscore_f1"] = f1_score
        summary_score["rouge1"] = rouge1
        summary_score["rouge2"] = rouge2
        summary_score["rougeL"] = rougeL
        summary_score["rougeLsum"] = rougeLsum
        total_questions += 1

        save_result_incrementally(result, csv_file_path, extraction_type)
        save_last_question_data(total_questions, summary_score, last_question_stats_file_path)

    avg_score = f1_score/total_questions if total_questions > 0 else 0
    print(f"Average f1 score for {extraction_type}: {avg_score:.4f}")
    return results

In [ ]:
def summarizer_fn(source_text, max_new_tokens=200):
    # Tokenize source, truncate to model’s max tokens.
    inputs = tokenizer(source_text, return_tensors="pt", truncation=True, max_length=tokenizer.model_max_length).to("cuda")

    # model.generate(do_sample=False, max_new_tokens=... )
    output = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)

    # Decode.
    generated_summary = tokenizer.batch_decode(output, skip_special_tokens=True)[0]

    return generated_summary

In [ ]:
from datasets import load_dataset

greedy_results = experiment(extraction_type="greedy") #we have a problem here, cause it gives us "May" instead of a number
usc_results = experiment(extraction_type="flexible", num_samples=num_samples)

# Summarization baselines
# gov_test = load_dataset("gov_report")["test"]
# gov_scores = run_summarization_baseline(
#     gov_test, source_key="report", ref_key="summary", summarizer_fn=run_greedy # Using the summarizer_fn defined earlier
# )
# print("GovReport (greedy) →", gov_scores)

# toe_results = {}
# for i, extraction_type in enumerate(EXTRACTION_TYPES_MATH[:1]):
#  toe_results[i] = experiment(extraction_type=extraction_type, num_samples=num_samples)

greedy answer: Harry Potter star Daniel Radcliffe gains access to a reported £20 million fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.
S {'rouge1': np.float64(0.3714285714285714), 'rouge2': np.float64(0.23529411764705882), 'rougeL': np.float64(0.34285714285714286), 'rougeLsum': np.float64(0.34285714285714286)}
{'question_num': 0, 'question': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Au

To use models from the Hugging Face Hub, you can set up your Hugging Face API token in Colab secrets.

Now we can load the Llama 3 model and tokenizer.